Search database for all CSV files.

In [16]:
import os
import pandas as pd
from m0_1_system_variables import test_data_path

Create a directory of source files:

In [17]:
complete_csv_path_list = []
for root, dirs, files in os.walk(test_data_path):
        for filename in files:
                if filename.lower().endswith('csv'):
                    complete_csv_path_list.append(r"{0}\{1}".format(root, filename))

Identify formatting of source files and import the data to buffer:

In [18]:
import chardet

data_type = []
# Determine the file encoding
for file_path in complete_csv_path_list:
    with open(file_path, 'rb') as f:
        result = chardet.detect(f.read())
    file_encoding = result['encoding']
    data_type.append(file_encoding)

In [19]:
print(data_type)

['ascii', 'ascii', 'ascii', 'ascii', 'ascii', 'ascii', 'ascii', 'ascii', 'ascii', 'ascii', 'ascii', 'ascii', 'ascii', 'ascii', 'ascii', 'ascii', 'ascii', 'ascii', 'ascii', 'ascii', 'Windows-1252', 'Windows-1252', 'ascii', 'ascii', 'ascii', 'ascii', 'ascii', 'ascii', 'ascii', 'ascii', 'ascii']


In [20]:
import pandas as pd

# Create an empty list to store the dataframes
df_list = []
vin_list = []
# Iterate through each file path in the list
for file_path in complete_csv_path_list:
    # Read the CSV file into a dataframe
    # Difficulty here: some files are ascii and some files are 'Windows-1252' encoding.
    with open(file_path, 'rb') as f:
        result = chardet.detect(f.read())
        detected_encoding = result['encoding']

    df = pd.read_csv(file_path, delimiter = '\r\n', encoding = detected_encoding, engine = 'python', names = ['Complete Record'])
    df_list.append(df)

In [21]:
df_list[:2]

[                                       Complete Record
 0                                 CDR FILE INFORMATION
 1                   User Entered VIN,1FTBR1Y8XMKA57198
 2                                         User,W Craig
 3                            Case Number,02951882-2023
 4    EDR Data Imaging Date,"02/10/2023             ...
 ..                                                 ...
 795                          Contains No Recorded Data
 796               LATERAL ACCELERATION (SECOND RECORD)
 797                          Contains No Recorded Data
 798                 VEHICLE ROLL ANGLE (SECOND RECORD)
 799                          Contains No Recorded Data
 
 [800 rows x 1 columns],
                                        Complete Record
 0                                       System:  SDM50
 1                                 CDR FILE INFORMATION
 2                   User Entered VIN,3GCNAAED6NG662964
 3                                   User,M.  Mikhailov
 4                   

In [22]:
master_import_dataframe = pd.DataFrame(columns=['Complete Records'])

for object in df_list:
    master_import_dataframe = pd.concat([master_import_dataframe, pd.DataFrame({'Complete Records': [object]})], ignore_index=True)


In [23]:
master_import_dataframe.head()

,Complete Records
0,Complet...
1,Complet...
2,Complet...
3,Complete...
4,Complet...


In [12]:
master_import_dataframe['Complete Records'][14].head()
test_cell = master_import_dataframe['Complete Records'][14].head()

In [14]:
#The dataframe is complete, but the CSV is truncated. Something is breaking here.
master_import_dataframe.to_csv('master_dataframe.csv', index=False)

In [16]:
master_import_dataframe["Complete Records"][24].to_csv('test_single_instance.csv', index=False)

In [12]:
master_import_dataframe_from_file = pd.read_csv('master_dataframe.csv')

In [13]:
master_import_dataframe_from_file.head()

,Complete Records
0,Complet...
1,Complete...
2,Complet...
3,Complete...
4,Complet...


pd will export single instances to CSV but does not adequately preserve the entire dataframe. Parquet did not function correctly, so pickle was chosen.

In [13]:
master_import_dataframe.to_pickle("master_import_dataframe.pkl")